In [3]:
### Just goofin here

from operator import index
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import SpanSelector
from scipy.optimize import curve_fit
import xlsxwriter
%matplotlib widget

### Reading the excel file
# df stands for dataframe (self-declared variable)
df = pd.read_excel(
    r"C:\Users\Home\Documents\School\Research\danielandabdulpvcdbap4.xlsx",
    # Do not read row 1 (zero index) of Excel file
    skiprows=[1],
    # Read cols A:C only in Excel
    usecols="A:C",
    # The index (or x-axis)
    index_col=0,
)

# Interpolating dataframe and getting rid of NaN
# .sort_index() numercally sorts index and all attached data
# .bfill() fills NaN values a particular value see documentation (only one Nan value after interpolate)
df = df.sort_index().interpolate().bfill()

# Offsetting load column to disregard initial tension on load cell 
# Uses avgLoad for the first two seconds
timeRange = df[(df.index) < 2]
avgLoad = timeRange["Load"].mean()
df = df - [avgLoad,0]

# Converting PVC from V to mV
df["PVC"] = df["PVC"] * 1000

# Converting Load from V to mv, then to grams (10mv = 250g Compression)
df["Load"] = df["Load"] * 1000 * 25

### This is a dataframe that stores the range we're interested in, 
# it is populated by the user selection from the graph
dfDomain = pd.DataFrame(columns=['xmin', 'xmax'])

# Telling matplotlib we want two plots and specifying their output size
fig, (graph1, graph2) = plt.subplots(2, figsize=(8, 6))

# Moving the plots so they're not so close together when outputting and covering labels
plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)

# Plotting graphs.......
graph1.plot(df["Load"])
graph2.plot(df["PVC"])

# Labeling graphs 
graph1.set_title("Applied Load")
graph2.set_title("Voltage across PVC Gel")
graph1.set_ylabel("Grams (g)")
graph2.set_ylabel("Millivolts (mV)")
graph2.set_xlabel("Time (s)")

# ---- Need to better understand how the functions below are defined

# This function is called everytime the SpanSelector widget is used 
def onselect_function1(min_value, max_value):
    dfDomain.loc[len(dfDomain.index)] = [min_value, max_value]
    print(dfDomain)
    return min_value, max_value


span1 = SpanSelector(
    graph1,
    onselect_function1,
    direction="horizontal",
    useblit=True,
    props=dict(alpha=0.5, facecolor="tab:blue"),
    interactive=True,
    drag_from_anywhere=True
)

TypeError: Cannot interpolate with all object-dtype columns in the DataFrame. Try setting at least one column to a numeric dtype.

In [ ]:
# 'row' in for loop always initializes at 0
for row in dfDomain.index:
    # Setting xmin and xmax for each iteration from dfDomain
    xmin = dfDomain["xmin"][row]
    xmax = dfDomain["xmax"][row]
    # Manipulating data to use for exponential fit

    # The ranges selected from the graph are now cutting down the main dataframe
    range = df.Load[(df.index > xmin) & (df.index < xmax)]
    dfGraphData = pd.DataFrame(range)
    dfGraphData["Time"] = dfGraphData.index

    t = dfGraphData["Time"]

    ### Reset index to 0
    dfGraphData = pd.DataFrame(range)
    dfGraphData.reset_index(inplace=True)

    # Offset so time starts @ 0
    dfGraphData = dfGraphData - [xmin, 0]

    # The exponential decay function
    def exp_decay(t, a, tau, c):
        return a * np.e ** (-t / tau) + c

    # Giving the curve fit a nudge by telling very roughly what a, b, c values it should expect
    p0 = [100, 0.25, 1]

    # Use scipy.optimize.curve_fit to fit parameters to noisy data
    popt, pcov = scipy.optimize.curve_fit(
        exp_decay, dfGraphData["Time"], dfGraphData["Load"], p0
    )
    fit_a, fit_tau, fit_c = popt

    # Sample exp_decay with optimized parameters
    y_fit = exp_decay(dfGraphData["Time"], fit_a, fit_tau, fit_c)

    # Scatter Plot
    fig, ax = plt.subplots(1)
    ax.scatter(
        dfGraphData["Time"],
        dfGraphData["Load"],
        alpha=0.6,
        marker=".",
        color="#2b730a",
        label="Load",
    )

    # Line of best fit
    ax.plot(dfGraphData["Time"], y_fit, color="#0d1452", label="Fit")
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("Load (g)")
    ax.legend()
    ax.set_title("(Graph " + str(dfDomain.index[row]+1) + ")\n" + "Load vs. Time")


    print("Graph", dfDomain.index[row]+1)
    print("y = ae^(-bt)+ c\n The a, b, and c parameters are: ")
    print(fit_a, 1 / fit_tau, fit_c, "\n")
